In [1]:
import scipy.stats
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import plotly.plotly as py
import plotly.graph_objs as go
py.sign_in('AriaRen','AtiibfXGar8FRpQvhnzD')

In [2]:
def PoissonPP(rt, Dx, Dy = None, Dz = None, seed = None):
    '''
    rt = rate or Poisson distribution
    Dx, Dy, Dz = the dimension of 3D array. 
    seed = seed variable for random_state in .rvs arguments (default = None)
    
    POISSONPP determines the number of events 'N' for a rectangular region,
    given the rate 'rt', the dimensions, 'Dx', 'Dy', 'Dz' and seed variable.
    Returns a <Nx3> NumPy array.
    
    '''
    
    if Dy == None:
        Dy = Dx
    if Dz == None:
        Dz = Dx
    if seed == None: 
        N = scipy.stats.poisson(rt*Dx*Dy*Dz).rvs()
        x = scipy.stats.uniform.rvs(loc = 0, scale = Dx, size = ((N, 1)))
        y = scipy.stats.uniform.rvs(loc = 0, scale = Dy, size = ((N, 1)))
        z = scipy.stats.uniform.rvs(loc = 0, scale = Dz, size = ((N, 1)))
    else:
        N = scipy.stats.poisson(rt*Dx*Dy*Dz).rvs(random_state=seed)
        x = scipy.stats.uniform.rvs(loc = 0, scale = Dx, size = ((N, 1)), random_state=seed)
        y = scipy.stats.uniform.rvs(loc = 0, scale = Dy, size = ((N, 1)), random_state=seed + 1)
        z = scipy.stats.uniform.rvs(loc = 0, scale = Dz, size = ((N, 1)), random_state=seed + 2)
        '''
        print('Dx = {}'.format(Dx))
        print('Dy = {}'.format(Dy))
        print('N = {}'.format(N))
        '''
    P = np.hstack((x, y, z))
    return(P)


In [73]:
def ThomasPP(rt, Dx, Dy, Dz, sigma, mu, seed = None):
    '''
    rt = rate or Poisson distribution
    Dx, Dy, Dz = the dimension of 3D array
    sigma = the standard deviation of Gaussian distribution surrounding parent points
    mu = generate the count for each Gaussian distribution following Poisson distribution
    seed = seed variable for random_state in .rvs arguments (default = None)
    
    THOMASPP generates multiple Gaussian distribution surrounding given parents points, 
    which are created by PoissonPP(). The sample size of Gaussian distribution is determined by 
    Poisson distribution 'mu', where the variance is determined by 'Sigma'.
    Returns a <Nx3> NumPy array.
    '''

    # Create a set of parent points form a Poisson(kappa)
    # distribution on the square region [0, Dx] * [0, Dx]
    
    if seed == None:
        parents = PoissonPP(rt, Dx, Dy, Dz, seed = None)
    else:    
        parents = PoissonPP(rt, Dx, Dy, Dz, seed = seed)
    print(seed)
    print('seed')
    # M is the number of parents
    M = parents.shape[0]
    # an empty list for the Thomas process points
    x = []
    y = []
    z = []
    # for each parent point
    for i in range(M):
        # determine a number of children according to a Poisson(mu) distribution
        parent_x = parents[i][0]
        parent_y = parents[i][1]
        parent_z = parents[i][2]
        pdf_x = scipy.stats.norm(loc = parent_x, scale = sigma)
        pdf_y = scipy.stats.norm(loc = parent_y, scale = sigma)
        pdf_z = scipy.stats.norm(loc = parent_z, scale = sigma)
        print(parent_x)
        print(pdf_x)
        
        # check if the seed arg exists.
        if seed == None:
            N = scipy.stats.poisson(mu).rvs()
            children_x = list(pdf_x.rvs(N))
            children_y = list(pdf_y.rvs(N))
            children_z = list(pdf_z.rvs(N))
        else:
            N = sum(scipy.stats.poisson(mu).rvs(size = 32, random_state = seed))
            children_x = list(pdf_x.rvs(N, random_state = (seed + i + 1)))
            children_y = list(pdf_y.rvs(N, random_state = (seed + i + 2)))
            children_z = list(pdf_z.rvs(N, random_state = (seed + i + 3)))
            print('N')
            print(N)
            print(children_x)
            
        
        # concate x y coordinates
        x = x + children_x
        y = y + children_y
        z = z + children_z
        
        
    x = np.array([x]).T
    y = np.array([y]).T
    z = np.array([z]).T

    P = np.hstack((x, y, z))
    return P

In [4]:
def xyzroi(xyzarray, xmin, xmax, ymin, ymax, zmin, zmax):
    '''
    xyzarray: A <Nx3> NumPy array with xyz coordinates.
    xmin, xman: the range in x-axis
    ymin, ymax: the range in y-axis
    zmin, zmax: the range in z-axis
    
    XYROI crop the dataset by given ranges in x and y axis ('xmin', 'xmax', 
    'ymin', 'max'), then return a <Nx2> NumPy array. 
    '''

    temp_xyz = xyzarray[(xyzarray[:,0] > xmin) & (xyzarray[:,0] < xmax) & (xyzarray[:,1] > ymin) & (xyzarray[:,1] < ymax)\
                       & (xyzarray[:,2] > zmin) & (xyzarray[:,2] < zmax)]
    return temp_xyz


In [5]:
def xyzroi_idx(xyzarray, xmin, xmax, ymin, ymax, zmin, zmax):
    '''
    xyzarray: A <Nx3> NumPy array with xy coordinates.
    xmin, xman: the range in x-axis
    ymin, ymax: the range in y-axis
    zmin, zmax: the range in z-axis
    
    XYZROI crop the dataset by given ranges in x, y and z axis ('xmin', 'xmax', 
    'ymin', 'ymax', 'zmin', 'zmax'), then return a <Nx3> NumPy array. 
    
    The last column is index. 
    '''

    idx = xyzarray.shape[0]
    idxarray = np.array([range(idx)]).T
    xyzarray = np.hstack((xyzarray, idxarray))
    temp_xyz = xyzarray[(xyzarray[:,0] > xmin) & (xyzarray[:,0] < xmax) & (xyzarray[:,1] > ymin) & (xyzarray[:,1] < ymax)\
                      & (xyzarray[:,2] > zmin) & (xyzarray[:,2] < zmax)]
    return temp_xyz

In [ ]:
a = ThomasPP(100, 2, 2, 2, 0.1, 0.8, seed = 2)

In [80]:
x,y,z = a.transpose()
trace1 = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=2,
        color=z,                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    )
)

data = [trace1]
layout = go.Layout(
    margin=dict(
        l=1,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='3d-scatter-colorscale')